# Lab 5. Time series classification

This time we will try to deal with time series for the classification task! There can be many different ways to deal with the problem. You can use traditional machine learning methods and regard one time point as one individual feature. Then you can use Decision Tree, Random Forests, and many more in scikit-learn. Or we can also use deep learning methods such as Fully Connected Network or Convolutional Neural Network. Finally, there are vast of special time series classification methods as an active research field.

In this lab, we will cover three different ways: 1) simple deep learning model, 2) complex models dedicated to time series classification with sktime, and 3) feature extraction methods with traditional scikit-learn model.

This lab will mainly focus on:
- Load and pre-process time series datasets with sktime
- Learn how to create simple deep learning models for time series classification
- Learn how to use sktime pipeline and extract important features
- Learn how to use the state-of-the-art time series classification methods with sktime

### Preparation

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd

Throughout the lab, we will use the famous **FordA** dataset from [the UCR time series repository](http://www.timeseriesclassification.com).

<img src="http://www.timeseriesclassification.com/images/datasets/FordA.png">

In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

X_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
X_test, y_test = readucr(root_url + "FordA_TEST.tsv")

The dataset has 3,601 instances with 500 time points each in its training set.

In [1]:
# check the shape of training and test sets!


And we are dealing with a binary classification task!

In [2]:
# check the unique labels!


### Section 1: Using tensorflow for time series classification

Before we get into special time series classification methods, we, again, can try tensorflow to handle it. The easiest way would be to use Fully Connected Network but you can use any type of network you want. Here let's bring our first neural network in Lab 3 and try to change it a bit to adjust our time series dataset.

First we need to fix few things of the dataset. This time we will use `validation_split` option without creating a validation set, then we need to shuffle the data ourselves. Of course you can also use `tf.Data` but this time we will skip it (If you use `tf.Data`, you cannot use validation_split option and thus need to create your own validation set). We will simply use `np.random.permutation` this time to shuffle the indices.

In [ ]:
# permute the indices by using np.random.permutation
idx = None
# get new shuffled training instances and labels
X_train = None
y_train = None

Next, as we checked before the labels are [-1, 1]. Unfortunately, tensorflow does not recognize negative number as a label so we need to fix it as well.

In [ ]:
# fix the index -1 to 0


Now we are ready to use the dataset! We are not creating a model specially dedicated to time series. However, we would like to use the same architecture on a different dataset. Then we need to change input and output layers! How can we do this?

In [ ]:
# model = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=[28, 28], name='flatten'), # FIX 1
#     keras.layers.Dense(300, activation='relu', name='dense1'),
#     keras.layers.Dense(100, activation='relu', name='dense2'),
#     keras.layers.Dense(10, activation='softmax', name='output') # FIX 2
# ])

# model.compile(loss="sparse_categorical_crossentropy", # FIX 3
#               optimizer="sgd", 
#               metrics=["accuracy"])

In [ ]:
# change the input and output layers of the model
model = None 

And we compile and fit the model.

In [ ]:
# compile the model with a new loss function


In [ ]:
# fit the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=30)

Now let's check the test accuracy score of our simple model. It is highly overfitted (but still fine...)! The best reported score is about 95%.

In [ ]:
model.evaluate(X_test, y_test)

Of course you can apply CNN, RNN, or many more including pretrained network by changing the same part of the network. This will remain as your task (and future Homework 3!).

### Section 2: Sktime basics: ROCKET, STSF, and HIVE-COTE

Now we move on to sktime. There are many libraries specially made for time series including [sktime](http://sktime.org), [pyts](https://pyts.readthedocs.io/en/stable/), and [wildboar](https://github.com/wildboar-foundation/wildboar), and most of them follow scikit-learn-style interface. However, sktime is the most active library with the biggest community. It has the most recent time series classification methods as well as some basic prediction methods too.

Unfortunately, sktime is not part of anaconda nor is supported by Colab by default, so you may need to install it if you haven't.

In [ ]:
!pip install sktime

#### Dataset with sktime

While we will use the same type of the dataset, let's check how we can load the dataset with sktime and what kind of format will be accepted.

First, if you use a UCR/UEA dataset (mainly for research purposes), you do not even need to download it yourself since sktime offers a convenient way to load it directly in its required format. The complete list of UCR/UEA datasets can be found [on their official website](http://www.timeseriesclassification.com).

In [ ]:
# load_UCR_UEA_dataset
from sktime.datasets import load_UCR_UEA_dataset

In [ ]:
# name: It's default format is Pandas DataFrame with numpy array as its member
X, y = None

In [ ]:
X

In [ ]:
# return_type: You can always change the format into a bit more general ones...
X, y = None

In [ ]:
X

In [ ]:
X.shape

In [ ]:
# Most universal format would be [numpy3D] if we generalize 'one dimension' as a multivariate time series dataset with dim = 1
X, y = None

In [ ]:
# NOTE that this would not be supported by tensorflow's RNN! - RNN has (instance, timestep, dimension)
X.shape

UCR datasets have pre-split training and test sets and you can even load it by putting an option.

In [ ]:
# split
X_train, y_train = None
X_test, y_test = None

However, if you need to deal with your own personal data, you can load it using scikit-learn as you have done. However, you need to make sure that it follows one of those three formats above to make sktime support the dataset without any problem.

In [ ]:
data = pd.read_csv('FordA_TRAIN.txt', header=None, delimiter='  ')

In [ ]:
data.head()

0         1         2         3         4         5         6         7    \
0 -1.0 -0.797172 -0.664392 -0.373015  0.040815  0.526936  0.984288  1.353120   
1  1.0  0.804855  0.634629  0.373474  0.038343 -0.340988 -0.740860 -1.109667   
2 -1.0  0.727985  0.111284 -0.499124 -1.068629 -1.578351 -1.990534 -2.302031   
3 -1.0 -0.234439 -0.502157 -0.732488 -0.946128 -1.139739 -1.323336 -1.490243   
4 -1.0 -0.171328 -0.062285  0.235829  0.710396  1.239969  1.649823  1.876321   

        8         9    ...       491       492       493       494       495  \
0  1.578108  1.659251  ...  1.120756  0.722417  0.362068  0.092083 -0.081268   
1 -1.395357 -1.570192  ...  0.386403  0.049213 -0.258138 -0.510583 -0.683647   
2 -2.503403 -2.585211  ...  0.394463  0.463685  0.507735  0.517174  0.504588   
3 -1.607077 -1.620430  ... -0.952804 -0.929437 -0.922761 -0.929437 -0.909409   
4  1.865535  1.703751  ...  0.776188  0.725496  0.697453  0.731967  0.808545   

        496       497       498       499       500  
0 -0.212573 -0.391456 -0.664392 -1.073796 -1.564343  
1 -0.773817 -0.785255 -0.714885 -0.560443 -0.319086  
2  0.476270  0.438513  0.394463  0.339400  0.255391  
3 -0.835970 -0.695768 -0.478790 -0.188707  0.119736  
4  0.839823  0.733046  0.437520 -0.026585 -0.602213  

[5 rows x 501 columns]

In [ ]:
data.to_numpy().shape #

(3601, 501)

However, let's stick to the easiest way this time :). One thing you may notice is that when we used sktime above, we only had one complete set. However, all UCR/UEA datasets have predefined train and test split and this can also be specified as an option.

In [ ]:
X_train, y_train = load_UCR_UEA_dataset(name="FordA", split="train", return_type="numpy2D")
X_test, y_test = load_UCR_UEA_dataset(name="FordA", split="test", return_type="numpy2D")

In [ ]:
X_train.shape, X_test.shape

#### DTW: The most basic but powerful method

Dynamic time warping is rather simple but powerful method for time series classification. Sktime supports both DTW and KNearestNeighbor classifier with DTW.

In [ ]:
from sktime.distances import dtw_distance, euclidean_distance

In [ ]:
dtw_distance(X_train[4], X_train[10]) # different class

In [ ]:
euclidean_distance(X_train[4], X_train[10]) # different class

In [ ]:
dtw_distance(X_train[1], X_train[10]) # same class

In [ ]:
euclidean_distance(X_train[1], X_train[10]) # same class

**KNN with DTW**: You can use the distance functions directly but sktime offers `KNeighborsTimeSeriesClassifier` as a built-in classifier that supports various distance measures including DTW and its variants. [Library document](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.classification.distance_based.KNeighborsTimeSeriesClassifier.html)

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

The usage is exactly the same as scikit-learn classifiers: **fit, predict,** and **score**!

In [ ]:
# different distances
knn_euc = None
knn_dtw = None

In [ ]:
knn_euc.fit(X_train, y_train)
knn_dtw.fit(X_train, y_train)

In [ ]:
knn_euc.score(X_test, y_test) # 66.5

Again, the best reported is about 95% and our very simple FCN model got about 80%. It seems like both DTW and Euclidean do not work very well to handle this dataset.

In [ ]:
knn_dtw.score(X_test, y_test) # EXTREMELY SLOW! - 66.5

Note that the distance function is also a parameter and the best distance might still depend on your task such as dataset. You can try grid search or randomized search to find out the most suitable distance measure for your task.

#### ROCKET: Fastest and probably the best

The simplest distance based kNN models did not give us satisfying results. Let's try another method before we move on to the next topic. Sktime has the most state-of-the-art time series classifiers as well and they are also quite easy to use. ROCKET (and its variants) is the fastest as you can guess by its name, and also provides quite powerful performance. [MiniRocket](https://www.sktime.net/en/stable/examples/transformation/minirocket.html) 

In [ ]:
from sktime.classification.kernel_based import RocketClassifier

In [ ]:
rocket = RocketClassifier() # SLOW!!!
rocket.fit(X_train, y_train)

In [ ]:
rocket.score(X_test, y_test) # 93.78

Other interesting classifiers supported by sktime are...: HIVE-COTE, BOSS, STSF, and also CNN!

### Section 3. Transformers: Feature extraction, processing, and sktime pipeline

#### Feature extraction

Another powerful way to deal with time series classification is to extract useful features from time series and regard them as a set of tabular features. Then you can fit any traditional methods (or even advanced methods such as deep learning or XGBoost) on those newly generated features. These feature extraction methods are part of scikit-learn's **transformer** interface.

The most recent feature extractor is [catch22](https://arxiv.org/pdf/1901.10200.pdf). This tries to get 22 most representative features to distinguish the classes out of time series values. 

In [ ]:
from sktime.transformations.panel.catch22 import Catch22

Now we can regard those features as 'features' that we have seen from tabular datasets. Those features no longer have temporal properties. This means that we can use any scikit-learn classifiers as usual.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# There is a bit of technical problem ... Catch22 does not handle numpy array type
from sktime.datatypes._panel._convert import from_2d_array_to_nested
# Or you can load the data again without specifying numpy2D format!

X_train_df = from_2d_array_to_nested(X_train)
X_test_df = from_2d_array_to_nested(X_test)

In [ ]:
catch22 = Catch22()
# fit_transform and transform
# As we always do when we standardize the data...
transformed_train = None
transformed_test = None

In [ ]:
X_train.shape

In [ ]:
transformed_train.shape

In [ ]:
X_test.shape

In [ ]:
transformed_test.shape

In [ ]:
rf = RandomForestClassifier()
rf.fit(transformed_train, y_train)
rf.score(transformed_test, y_test)

**ROCKET** can also be used as a transformer (if you remember how it works)!

In [ ]:
from sktime.transformations.panel.rocket import Rocket
#from sktime.classification.kernel_based import RocketClassifier

You can put the new features into any classifier. **RocketClassifier** is actually Rocket transformer + linear regressor. However of course you can use those transformed features with another classifier such as Random Forest.

In [ ]:
rocket = Rocket() # SLOW!!!
transformed_train = rocket.fit_transform(X_train_df)
transformed_test = rocket.transform(X_test_df)  

In [ ]:
rf = RandomForestClassifier()
rf.fit(transformed_train, y_train)
rf.score(transformed_test, y_test)

There are much more transformers such as temporal abstractions (SAX mainly), shapelets, and matrix profiles.

#### Processing

Another way to transform the dataset is to change the values of the data (so it's closer to the actual meaning of 'transformer' in scikit-learn.). However, those are quite similar to simply applying NumPy operation if your data type is ndarray - and relatively slower! This type of transformation is more often used for prediction tasks such as [Box-Cox transformation](https://builtin.com/data-science/box-cox-transformation-target-variable). 

In [ ]:
# EXPONENTIAL
from sktime.transformations.series.exponent import ExponentTransformer

In [ ]:
square = ExponentTransformer(power=2)

In [ ]:
transformed_train = square.fit_transform(X_train_df)
transformed_test = square.transform(X_test_df)  

Unlike feature extraction transformers above, these transformers still keep temporal dependencies. You can of course use any classifier in scikit-learn than in this case you are regarding time series as tabular dataset ignoring temporal properties. Of course you can apply several transformers in order.

In [ ]:
rocket = RocketClassifier()
rocket.fit(transformed_train, y_train)
rocket.score(transformed_test, y_test)

#### Pipeline

As we saw before in Lab 1, we can merge those preprocessing steps and classifiers in one wrapper called **pipeline**. All sktime's methods are well compatible with scikit-learn's pipeline, while sktime itself also has its own pipeline method.

In [ ]:
# "magic multiplication" - clever way of constructing a pipeline
pipe_sktime = None
pipe_sktime.fit(X_train_df, y_train)
pipe_sktime.score(X_test_df, y_test)

However, this sktime wrapper does not support GridSearch and RandomizedSearch. If you would like to find a parameter, (officially) you need to use scikit-learn's pipeline. However, you can use scikit-learn's GridSearch (or RandomizedSearch) by calling sktime's ClassifierPipeline directly not using "magic multiplication".

In [ ]:
from sktime.classification.compose import ClassifierPipeline

In [ ]:
# The logic is a bit different -> estimator(), [list of transformers with names]
pip = None

In [ ]:
param_grid = None

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
grid = RandomizedSearchCV(pip, param_grid, n_iter = 3, verbose = 2)

In [ ]:
grid.fit(X_train_df, y_train)

In [ ]:
grid.best_estimator_, grid.best_score_

### Task 1: Deep learning based time series classification methods

There has been several neural network models dedicated to time series classification. Besides your own models that you developed in Lab 4, now you will develop such dedicated models by referring to some papers, and test if they indeed perform better than your rough models. There are two famous papers as follows:
 - [Convolutional neural networks for time series classification (2017)](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7870510)
 - [Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2017)](https://arxiv.org/abs/1611.06455)

First paper's idea is already implemented in sktime, with the name `CNNClassifier`. Second paper has three models and those are easy to develop using tensorflow. Now the task is to develop two models (MLP and FCN) in the second paper and test it together with `CNNClassifier`.

Use the same **FordA** dataset, and test sktime's `CNNClassifier` and MLP and FCN models you develop. Report the test scores of three models. For MLP and FCN, you may need to satisfy the following requirement:

- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- You should run the model at least 10 epochs.
- You do not need to use `tf.Data` this time but it is still recommended to use it.
- For `CNNClassifier`, you can run it with the default parameters or reduce the number of epoch (default is 2000).
- Please use the predefined test dataset to report the test scores.

Note that the main purpose of this task is to check if you can develop a similar network structure with description. If the detail of the specific part (e.g., size of one layer or some custom parameters like epoch) is missing in the paper, you can set it on your own.
 

### Task 2: Randomized search and feature extraction with traditional methods


As we saw above, we can use RandomizedSearch to find out optimal parameter options. However, it is not well supporting scikit-learn's classifiers such as DecisionTree or RandomForest. However, as we tried, it is also possible to use the output of sktime transformer (e.g., catch22) to train scikit-learn models such as RandomForest. We might want to put them together in one pipeline. Sktime supports this with `SklearnClassifierPipeline` and this is your second task to use it.

Pick one classifier from scikit-learn (that can be anything: Decision Tree, Logistic Regressor ...) and two transformers from sktime and create `SklearnClassifierPipeline`. That can be Rocket with RandomForest or Catch22 with DecisionTree as we tried in this lab. Pick two parameters from each module (in total six, two from classifier and four from two transformers) and run Randomized search on the pipeline you define and report the test score of the best model found by Randomized Search. Compare your best score with the score from 1) the same model with default setting, 2) the models from Task 1.

Note that you do not need to achieve better performance by performing randomized search for this task (but still good to try!).